In [1]:
import ee
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import rasterio

ee.Authenticate()
ee.Initialize(project='geototh-learning1')

In [39]:
modis_collection = ee.ImageCollection("MODIS/061/MOD09GA").filterDate('2023-12-01','2024-01-01')
co_bbox = ee.Geometry.BBox(-109.059196,36.992751,-102.042126,41.001982)
ds = xr.open_dataset(modis_collection,
                     engine='ee',
                     crs='EPSG:4326',
                     scale=.01,
                     geometry=co_bbox)

df = ds.to_dataframe()
df = df.reset_index(level = ['lon','lat','time'])
IC_gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)
bands = ['time','sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04',
       'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07','geometry','lon','lat']
IC_gdf = IC_gdf[bands]
IC_gdf.to_parquet("C:\\Users\\Chris\\co_cloudcover\\data\\raw\\ImageCollection_202312.parquet")


### **Export GeoDataFrame as GeoTiff**
#### Define functions for gtiff export

In [36]:
from rasterio.transform import Affine
import math

def calculate_affine(coeffs):
    """https://trac.osgeo.org/postgis/wiki/DevWikiAffineParameters"""
    sx,sy,tx,ty = coeffs['sx'],coeffs['sy'],coeffs['tx'],coeffs['ty']
    θ,kx,ky = coeffs['θ'],coeffs['kx'],coeffs['ky']
    
    o11 = sx * ((1 + kx*ky) * math.cos(θ) + ky*math.sin(θ) )
    o12 = sx * ( kx*math.cos(θ) + math.sin(θ) )
    o21 = -sy * ( -(1 + kx*ky) * math.sin(θ) + ky*math.cos(θ) )
    o22 = sy * ( -kx*math.sin(θ) + math.cos(θ) )

    return(Affine(o11,o12,tx,o21,o22,ty))

def rbgArray_to_RGBgeotiff(rgb,f_name,directory,profile):

    with rasterio.open(directory + f_name,'w',**profile) as dst:
        dst.write(rgb)

    return(None)
    

def make_RGBarray(gdf,img_date,rgb_bands):

    #select data by date:
    date_gdf = gdf[gdf['time']==img_date]
    
    # Select Bands:
    r_band = rgb_bands["red"]
    g_band = rgb_bands["green"]
    b_band = rgb_bands["blue"]
    rgb_gdf = date_gdf[[r_band,g_band,b_band,"geometry"]]
    
    # Extract Geometry:
    rgb_gdf['x'],rgb_gdf['y'] = rgb_gdf.geometry.x,rgb_gdf.geometry.y
    dim_x,dim_y = len(set(rgb_gdf.x)),len(set(rgb_gdf.y))

    # Construct channel arrays
    r_array = rgb_gdf[r_band].to_numpy().reshape((dim_x,dim_y))
    g_array = rgb_gdf[g_band].to_numpy().reshape((dim_x,dim_y))
    b_array = rgb_gdf[b_band].to_numpy().reshape((dim_x,dim_y))

    #Construct RGB Array
    rgb = np.stack((r_array,g_array,b_array))
    
    return(rgb,dim_x,dim_y)

#### Write snow-enhanced gtiffs

In [34]:
rgb_bands = {
    "profile_name" : "snow_enhanced",
    "red":"sur_refl_b01",
    "green":"sur_refl_b05",
    "blue":"sur_refl_b07",
    }

img_dates = list(set(IC_gdf["time"]))
for img_date in img_dates:
    img_date = img_date.strftime("%Y-%m-%d")
    
    rgb,dim_x,dim_y = make_RGBarray(IC_gdf,img_date,rgb_bands)
    
    transform_coeffs = {
        'sx':abs((min(IC_gdf['lon']) - max(IC_gdf['lon']))/dim_y)  ,# scale factor in x direction
        'sy':abs((min(IC_gdf['lat']) - max(IC_gdf['lat']))/dim_x)  ,# scale factor in y direction
        'tx':min(IC_gdf['lon'])  ,# offset in x direction
        'ty':min(IC_gdf['lat'])  ,# offset in y direction
        'θ':math.pi/2   ,# angle of rotation clockwise around origin
        'kx':0  ,# shearing parallel to x axis
        'ky':0  ,# shearing parallel to y axis 
        }
    
    rgb_profile = {
        'driver':'GTiff',
        "width": dim_x,
        "height": dim_y,
        "count": 3,
        "dtype": rgb.dtype,
        "crs": "EPSG:4326",
        "transform": calculate_affine(transform_coeffs),
        "nodata": np.nan,
        }
    
    fig_dir = "C://Users//Chris//co_cloudcover//reports//figures//"
    f_name = f"{rgb_bands['profile_name']}_{img_date}.tif"
    rbgArray_to_RGBgeotiff(rgb,f_name,fig_dir,rgb_profile)

C:\Users\Chris\anaconda3\envs\sensat\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Chris\anaconda3\envs\sensat\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Chris\anaconda3\envs\sensat\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

#### Write true color gtiffs

In [37]:
rgb_bands = {
    "profile_name" : "true_color",
    "red":"sur_refl_b01",
    "green":"sur_refl_b04",
    "blue":"sur_refl_b03",
    }

img_dates = list(set(IC_gdf["time"]))
for img_date in img_dates:
    img_date = img_date.strftime("%Y-%m-%d")
    
    rgb,dim_x,dim_y = make_RGBarray(IC_gdf,img_date,rgb_bands)
    
    transform_coeffs = {
        'sx':abs((min(IC_gdf['lon']) - max(IC_gdf['lon']))/dim_y)  ,# scale factor in x direction
        'sy':abs((min(IC_gdf['lat']) - max(IC_gdf['lat']))/dim_x)  ,# scale factor in y direction
        'tx':min(IC_gdf['lon'])  ,# offset in x direction
        'ty':min(IC_gdf['lat'])  ,# offset in y direction
        'θ':math.pi/2   ,# angle of rotation clockwise around origin
        'kx':0  ,# shearing parallel to x axis
        'ky':0  ,# shearing parallel to y axis 
        }
    
    rgb_profile = {
        'driver':'GTiff',
        "width": dim_x,
        "height": dim_y,
        "count": 3,
        "dtype": rgb.dtype,
        "crs": "EPSG:4326",
        "transform": calculate_affine(transform_coeffs),
        "nodata": np.nan,
        }
    
    fig_dir = "C://Users//Chris//co_cloudcover//reports//figures//"
    f_name = f"{rgb_bands['profile_name']}_{img_date}.tif"
    rbgArray_to_RGBgeotiff(rgb,f_name,fig_dir,rgb_profile)

C:\Users\Chris\anaconda3\envs\sensat\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Chris\anaconda3\envs\sensat\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Chris\anaconda3\envs\sensat\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [ ]:
sns.pairplot(IC_gdf)